In [ ]:
data = np.load("wang2020_iddd_test_df.npz")
train_df_dataframe1 = pd.DataFrame.from_dict({item: data[item] for item in data.files}) #.sort_values('dolos_file_num')
data.close()
print(train_df_dataframe1)

         preds  file_names
0     0.869222         0.0
1     0.889062         1.0
2     0.249400        10.0
3     0.634820       100.0
4     0.801293      1000.0
...        ...         ...
1495  0.941792       995.0
1496  0.301538       996.0
1497  0.737886       997.0
1498  0.651643       998.0
1499  0.685391       999.0

[1500 rows x 2 columns]


In [ ]:
import pandas as pd
import numpy as np

experiment = "iddd_train_df"

data = np.load("dolos_" + experiment + ".npz")
train_df_dataframe1 = pd.DataFrame.from_dict({item: data[item] for item in data.files}) #.sort_values('dolos_file_num')
data.close()

data = np.load("mesonet_" + experiment + ".npz")
train_df_dataframe2 = pd.DataFrame.from_dict({item: data[item] for item in data.files}) #.sort_values('dolos_file_num')
data.close()

final_train_df = pd.merge(train_df_dataframe1, train_df_dataframe2, left_on='dolos_file_num', right_on='mesonet_file_num', how='left')
final_train_df = final_train_df.drop(['dolos_file_num', 'mesonet_file_num', 'mesonet_actual'], axis=1)
print(final_train_df)

      dolos_preds  dolos_actual  mesonet_preds
0             0.0           1.0            1.0
1             0.0           1.0            0.0
2             0.0           1.0            0.0
3             0.0           1.0            1.0
4             0.0           1.0            0.0
...           ...           ...            ...
2495          0.0           1.0            0.0
2496          0.0           1.0            0.0
2497          0.0           1.0            1.0
2498          0.0           1.0            1.0
2499          0.0           1.0            1.0

[2500 rows x 3 columns]


### Naive Emsemble Model

Outputs 1 whenever one of the models output 1

In [ ]:
final_train_df['ensemble_preds'] = final_train_df.apply(lambda x: 1.0 if (x['dolos_preds'] == 1.0 or x['mesonet_preds'] == 1.0) else 0.0, axis=1)
# np.where(final_train_df['dolos_preds'] == 1.0 or final_train_df['mesonet_preds'] ==?
print(final_train_df)

      dolos_preds  dolos_actual  mesonet_preds  ensemble_preds
0             0.0           0.0            1.0             1.0
1             0.0           0.0            0.0             0.0
2             0.0           0.0            0.0             0.0
3             0.0           0.0            0.0             0.0
4             0.0           0.0            0.0             0.0
...           ...           ...            ...             ...
1495          0.0           0.0            0.0             0.0
1496          0.0           0.0            1.0             1.0
1497          0.0           0.0            1.0             1.0
1498          0.0           0.0            0.0             0.0
1499          0.0           0.0            1.0             1.0

[1500 rows x 4 columns]


In [ ]:
from sklearn.metrics import average_precision_score, accuracy_score
print(accuracy_score(final_train_df["dolos_actual"], final_train_df["ensemble_preds"]))

0.6213333333333333


### AdaBoost

In [ ]:
X_train = final_train_df[['dolos_preds', 'mesonet_preds']]
y_train = final_train_df['dolos_actual']

In [ ]:
from sklearn.ensemble import AdaBoostClassifier
# from sklearn.model_selection import cross_val_score

clf = AdaBoostClassifier(n_estimators=20, algorithm="SAMME",)

model = clf.fit(X_train, y_train)

/usr/local/lib/python3.11/dist-packages/sklearn/ensemble/_weight_boosting.py:519: FutureWarning: The parameter 'algorithm' is deprecated in 1.6 and has no effect. It will be removed in version 1.8.
  warnings.warn(


In [ ]:
data = np.load("dolos_iddd_test_df.npz")
test_df_dataframe1 = pd.DataFrame.from_dict({item: data[item] for item in data.files}) #.sort_values('dolos_file_num')
data.close()

data = np.load("mesonet_iddd_test_df.npz")
test_df_dataframe2 = pd.DataFrame.from_dict({item: data[item] for item in data.files}) #.sort_values('dolos_file_num')
data.close()

final_test_df = pd.merge(test_df_dataframe1, test_df_dataframe2, left_on='dolos_file_num', right_on='mesonet_file_num', how='left')
final_test_df = final_test_df.drop(['dolos_file_num', 'mesonet_file_num', 'mesonet_actual'], axis=1)
print(final_test_df)

      dolos_preds  dolos_actual  mesonet_preds
0             0.0           1.0            1.0
1             0.0           1.0            0.0
2             0.0           1.0            0.0
3             0.0           1.0            0.0
4             0.0           1.0            1.0
...           ...           ...            ...
1495          0.0           1.0            1.0
1496          0.0           1.0            0.0
1497          0.0           1.0            1.0
1498          0.0           1.0            1.0
1499          0.0           1.0            0.0

[1500 rows x 3 columns]


In [ ]:
X_test = final_test_df[['dolos_preds', 'mesonet_preds']]
y_test = final_test_df['dolos_actual']

#Predict the response for test dataset
y_pred = model.predict(X_test)

In [ ]:
from sklearn.metrics import average_precision_score, accuracy_score
print(accuracy_score(y_test, y_pred))

1.0


In [ ]:
print(np.unique(y_pred))

[1.]
